## Intilization

In [1]:
import os
import itertools
import sys
from math import factorial as fac
sys.path.append("D:/TU_Delft/Msc_Building_Technology/Semester_3/Graduation/Aditya_Graduation_Project_BT/06_Libraries")
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
np.random.seed(0)
np.set_printoptions(threshold=sys.maxsize)
import networkx as nx
import pickle

## Load Lattice availability

In [2]:
lattice_path = os.path.relpath('Base_lattice_2.csv')
avail_lattice_base = tg.lattice_from_csv(lattice_path)
avail_lattice = avail_lattice_base*1
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)

## Create Stencil 

In [3]:
# creating neighborhood definition
stencil_von_neumann = tg.create_stencil("von_neumann", 1, 1)
stencil_von_neumann.set_index([0,0,0], 0)
#print(stencil_von_neumann)

## Create graph 

In [4]:
# find the number of all voxels
vox_count = avail_lattice.size 

# initialize the adjacency matrix
adj_mtrx = np.zeros((vox_count,vox_count))

# extract the neighbourhood of all voxels
# all_vox_neighs = avail_lattice.find_neighbours(stencil)

# Finding the index of the available voxels in avail_lattice
avail_index = np.array(np.where(avail_lattice == 1)).T

# fill the adjacency matrix using the list of all neighbours
for vox_loc in avail_index:
    # find the 1D id
    vox_id = np.ravel_multi_index(vox_loc, avail_lattice.shape)
    # retrieve the list of neighbours of the voxel based on the stencil
    vox_neighs = avail_lattice.find_neighbours_masked(stencil_von_neumann, loc = vox_loc)
    # iterating over the neighbours
    for neigh in vox_neighs:
        # setting the entry to one
        adj_mtrx[vox_id, neigh] = 1.0

# construct the graph 
g = nx.from_numpy_array(adj_mtrx)

## Create distance matrix

In [5]:
#dist_mtrx = nx.floyd_warshall_numpy(g)

In [6]:
#dist_mtrx = pickle.dump( dist_mtrx, open( "dist_matrix.p", "wb" ) )

In [7]:
pickle_matrix = pickle.load( open( "dist_matrix.p", "rb" ) ) 

## Create a list of points to create graphs 

In [21]:
flattened_lattice = avail_lattice.flatten()
all_indexes1D= np.array(flattened_lattice.nonzero())
all_indexes1D_tuple = flattened_lattice.nonzero()
all_indexes1D_list = []
for sublist in all_indexes1D_tuple:
    for item in sublist:
        all_indexes1D_list.append(item)
len(all_indexes1D_list)

684

In [20]:
## take out 3d arry where --1 then filter out themiddle segment and run the iterations

array([[ 243,  244,  245,  246,  247,  248,  249,  250,  251,  254,  255,
         256,  257,  258,  259,  260,  261,  262,  265,  266,  267,  268,
         269,  270,  271,  272,  273,  276,  277,  278,  279,  280,  281,
         282,  283,  284,  287,  288,  289,  290,  291,  292,  293,  294,
         295,  298,  299,  300,  301,  302,  303,  304,  305,  306,  309,
         310,  311,  312,  313,  314,  315,  316,  317,  320,  321,  322,
         323,  324,  325,  326,  327,  328,  331,  332,  333,  334,  335,
         336,  337,  338,  339,  342,  343,  344,  345,  346,  347,  348,
         349,  350,  353,  354,  355,  356,  357,  358,  359,  360,  361,
         364,  365,  366,  367,  368,  369,  370,  371,  372,  375,  376,
         377,  378,  379,  380,  381,  382,  383,  386,  387,  388,  389,
         390,  391,  392,  393,  394,  397,  398,  399,  400,  401,  402,
         403,  404,  405,  408,  409,  410,  411,  412,  413,  414,  415,
         416,  419,  420,  421,  422, 

## Create iterations

In [26]:
# iteration of all possible 5 nodes
all_permutations_nodes = (list(itertools.permutations(all_indexes1D_list, 3)))



[(0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 3),
 (3, 0),
 (3, 1),
 (3, 2)]

In [27]:
## indexing to check distances 
all_permutations_indexes = (list(itertools.permutations(range(5), 2)))
all_permutations_indexes

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 0),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 0),
 (2, 1),
 (2, 3),
 (2, 4),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 4),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3)]

In [13]:
all_valid_permutations=[]

for perm, index in zip(all_permutations_nodes,all_permutations_indexes):
    print (perm)
    print (index)

(243, 244, 245)
(0, 1)
(243, 244, 246)
(0, 2)
(243, 244, 247)
(0, 3)
(243, 244, 248)
(1, 0)
(243, 244, 249)
(1, 2)
(243, 244, 250)
(1, 3)
(243, 244, 251)
(2, 0)
(243, 244, 254)
(2, 1)
(243, 244, 255)
(2, 3)
(243, 244, 256)
(3, 0)
(243, 244, 257)
(3, 1)
(243, 244, 258)
(3, 2)


## Conditional statements for checking the validity 


In [24]:
'''
x=[]
for i, j, k, l,m in itertools.product(range(243,1142), range(243,1142),range(243,1142),range(243,1142),range(243,1142)):
    if pickle_matrix[i][j]> 10:
    elif pickle_matrix[i][j]> 10:
    elif pickle_matrix[i][k]> 10:
    elif pickle_matrix[i][l]> 10:
    elif pickle_matrix[i][m]> 10:
        x.append(i)
    

## Find out set of possible nodes

In [25]:
x (quadratic assignment)

[[243, 274],
 [243, 284],
 [243, 285],
 [243, 294],
 [243, 295],
 [243, 296],
 [243, 304],
 [243, 305],
 [243, 306],
 [243, 307],
 [243, 314],
 [243, 315],
 [243, 316],
 [243, 317],
 [243, 318],
 [243, 324],
 [243, 325],
 [243, 326],
 [243, 327],
 [243, 328],
 [243, 329],
 [243, 334],
 [243, 335],
 [243, 336],
 [243, 337],
 [243, 338],
 [243, 339],
 [243, 340],
 [243, 344],
 [243, 345],
 [243, 346],
 [243, 347],
 [243, 348],
 [243, 349],
 [243, 350],
 [243, 351],
 [243, 352],
 [243, 354],
 [243, 355],
 [243, 356],
 [243, 357],
 [243, 358],
 [243, 359],
 [243, 360],
 [243, 361],
 [243, 362],
 [243, 363],
 [243, 364],
 [243, 365],
 [243, 366],
 [243, 367],
 [243, 368],
 [243, 369],
 [243, 370],
 [243, 371],
 [243, 372],
 [243, 373],
 [243, 374],
 [243, 375],
 [243, 376],
 [243, 377],
 [243, 378],
 [243, 379],
 [243, 380],
 [243, 381],
 [243, 382],
 [243, 383],
 [243, 384],
 [243, 385],
 [243, 386],
 [243, 387],
 [243, 388],
 [243, 389],
 [243, 390],
 [243, 391],
 [243, 392],
 [243, 393],

## Simulate the agents for all possible nodes

## Linear Assignment 

## Verification